<div dir="rtl">
       <h1>تبدیل فایل های IPYNB به HTML و استقرار خودکار در github-pages (بخش دوم)</h1>
</div><br/>

<div dir="rtl">
       به بخش دوم مقاله خوش اومدید. تو این بخش قراره ببینیم که چطوری میشه با استفاده از گیت هاب اکشن از اسکریپتی که تو <a href="https://vrgl.ir/oAqXx" target="_blank" rel="noopener nofollow">بخش اول این مقاله</a> نوشتیم به صورت خودکار استفاده کنیم.
</div><br/>

<div dir="rtl">
      نحوه عملکرد این اکشن به این شکله که شما هربار تغییراتی رو روی <strong>مخزن (repository)</strong> موردنظر پوش میکنید، چک میکنه که آیا فایل های با فرمت <strong>ipynb </strong>تغییر کردند یا خیر. اگر تغییری در اونا صورت نگرفته باشه که اتفاقی نمیوفته ولی اگه تغییر کرده باشن اسکریپتی که تو بخش اول نوشتیم رو اجرا میکنه. یعنی فایل های نوت بوک رو به فایل های <strong>HTML </strong>تبدیل میکنه و اونا رو به دایرکتوری تعیین شده برای <a href="https://pages.github.com/" target="_blank" rel="noopener nofollow">github pages</a> منتقل میکنه.
</div><br/>

<div dir="rtl">
       <h2>سرویس github actions چیه؟</h2>
</div><br/>

<div dir="rtl">
       هدف این سرویس اتوماتیک کردن برخی فرایندهاست، مثل تست، استقرار (deploy) و ...؛ گیت هاب اکشن <strong>رویداد محور (event-driven)</strong> هستش، یعنی زمانی که رویداد خاصی مثل پوش اتفاق بیوفته یک سری کارهایی که تعیین کردیم رو انجام میده.
</div><br/>

<div dir="rtl">
      اگه دوست دارید بیشتر با این سرویس آشنا بشید میتونید به <a href="https://docs.github.com/en/actions" target="_blank" rel="noopener nofollow">داکیومنت اصلی گیت هاب اکشن</a> مراجعه کنید.
</div><br/>

<hr style="background-color: floralwhite;" />

<div dir="rtl">
       اکشنی که قراره بنویسیم رو میتونین از <a href="https://github.com/iranipy/iranipy" target="_blank" rel="noopener nofollow">مخزن اصلی گیت هاب ایرانی پای</a> بردارید.
       <ul><li><a href="https://github.com/iranipy/iranipy/blob/main/projects/ipynb_docs_renderer/github-action/ipynb_docs_renderer.yml" target="_blank" rel="noopener nofollow">مشاهده اکشن</a></li><li><a href="https://github.com/iranipy/iranipy/tree/main/projects/ipynb_docs_renderer" target="_blank" rel="noopener nofollow">مشاهده پروژه</a> (شامل همه کدهای نوشته شده و فایلهای راهنما)</li></ul>
</div><br/>

<hr style="background-color: floralwhite;" />

<div dir="rtl">
    <h2>جریان کار (workflow)</h2>
</div><br/>

<div dir="rtl">
    برای استفاده از این ویژگی باید از یک سری قوانین پیروی کنیم. یکی از این قوانین قرار دادن فایل اکشن تو دایرکتوری ای هست که خود گیت هاب گفته. اول از همه لازمه که یک دابرکتوری به اسم <strong>github.</strong> تو مخزنمون ایجاد کنیم. حالا تو این دایرکتوری یک دایرکتوری دیگه به اسم <strong>workflows </strong>درست میکنیم. حالا تو این دایرکتوری یک فایل به اسم <strong>ipynb_docs_renderer.yml </strong>میسازیم. اسم فایل دلخواهه و قراره جلوتر ازش استفاده کنیم ولی پسوندش باید حتما yml باشه.
</div><br/>

<div dir="rtl">
   پس ساختار <strong>مخزن (repository)</strong> اینجوری شد:
</div><br/>

```
./repository/
│
├── .github/
     ├── workflows/
         ├── ipynb_docs_renderer.yml
```

<div dir="rtl">
    <h2>اکشن (action)</h2>
</div>

```
name: ipynb documents renderer
on:
  push:
    paths:
       - '**.ipynb'
jobs:
  build:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: [3.8]
  steps:
     - uses: actions/checkout@v2
     - name: Set up Python ${{ matrix.python-version }}
       uses: actions/setup-python@v2
         with:
           python-version: ${{ matrix.python-version }}
        - name: install dependencies
        run: |
          python -m pip install --upgrade pip
          python ipynb_docs_renderer.py install-dependencies
       - name: convert ipynb documents to html
       run: |
          python ipynb_docs_renderer.py render    
      - name: commit and push changes
       run: |
         python ipynb_docs_renderer.py commit-push
```

<div dir="rtl">
       این محتوای فایل <strong>yml </strong>هستش که تو مرحله قبل ساختیم. خب بریم ببینیم هر قسمتش چه کاری انجام میده.
</div><br/>

<div dir="rtl">
       <strong>name</strong>
</div><br/>

<div dir="rtl">
       کاملا اختیاریه. اسمیه که تو تب <strong>actions </strong>مخزن تو گیت هاب نشون میده.
</div><br/>

<div dir="rtl">
       <strong>on</strong>
</div><br/>

<div dir="rtl">
       رویدادی که میخوایم بعد از اتفاق افتادنش <strong>job </strong>اجرا بشه رو اینجا مینویسیم. اینجا ما نوشتم <strong>push </strong>پس سر هر بار پوش شدن <strong>job </strong>اجرا میشه. ولی یک شرط دیگه هم اضافه کردیم: <strong>paths</strong>. اینجا مشخص میکنیم که اگه کدوم دایرکتوری ها یا فایل ها تغییر کردن <strong>job </strong>رو اجرا کنه. اینجا نوشتیم <strong>ipynb.**</strong> که یعنی تو هر دایرکتوری و ساب دارکتوری ای فایلی با فرمت <strong>ipynb </strong>تغییر کرد <strong>job </strong>رو اجرا کن.
</div><br/>

<div dir="rtl">
       <strong>jobs</strong>
</div><br/>

<div dir="rtl">
       تمام کارایی که قراره با هر بار اجرا شدن این اکشن انجام بشه رو اینجا مینویسیم.
</div><br/>

<div dir="rtl">
       <strong>build</strong>
</div><br/>

<div dir="rtl">
      اطلاعات سیستم عامل و نسخه پایتونی که میخوایم برای این <strong>job </strong>استفاده کنیم رو اینجا مینویسیم.
</div><br/>

<div dir="rtl">
       <strong>runs-on</strong>
</div><br/>

<div dir="rtl">
       اینجا مشخض میکنیم که از چه سیستم عاملی و چه نسخه ای از اون سیستم عامل میخوایم استفاده کنیم. ما از <strong>ubuntu-latest</strong> استفاده کردیم، یعنی آخرین نسخه سیستم عامل ابونتو. امکان انتخاب چند تا سیستم عامل هم وجود داره.
</div><br/>

<div dir="rtl">
       <strong>strategy</strong>
</div><br/>

<div dir="rtl">
      تو این بخش نسخه پایتونی که میخوایم استفاده کنیم رو مشخض کردیم. میتونید برای هر <strong>job </strong>چند تا نسخه مختلف پایتون هم در نظر بگیرید.
</div><br/>

<div dir="rtl">
       <strong>steps</strong>
</div><br/>

<div dir="rtl">
       تمامی دستوراتی که قراره اجرا بشن رو اینجا مینویسیم. هر آیتمی که اینجا بنویسیم تو یک <strong>shell</strong> جداگونه اجرا میکنه.
</div><br/>

<div dir="rtl">
       <strong>uses: actions/checkout@v2 </strong>
</div><br/>

<div dir="rtl">
       اینجا گفتیم که از نسخه شماره 2 اکشن ها استفاده کن. این اکشن با دانلود کردن <strong>مخزن (repository) </strong>موردنظر، اجازه میده که کارای مختلف رو کدهای موجود تو اون <strong>مخزن (repository) </strong>انجام بدیم. همیشه از همین نسخه باید استفاده بشه (طبق اسناد موجود در سایت گیت هاب).
</div><br/>

<div dir="rtl">
       <strong>name: Set up Python ${{ matrix.python-version }} </strong>
</div><br/>

<div dir="rtl">
       اسم <strong>step </strong>ای که میخوایم اجرا کنیم رو اینجوری مشخص میکنیم. <strong>{{ matrix.python-version }}</strong> معادل نسخه پایتونی هست که تو <strong>matrix </strong>مشخص کردیم.
</div><br/>

<div dir="rtl">
       <strong>uses: actions/setup-python@v2</strong><br>
</div><br/>

<div dir="rtl">
       این اکشن یک محیط پایتون برامون فراهم میکنه.
</div><br/>

<div dir="rtl">
       <strong>with: python-version: ${{ matrix.python-version }} </strong>
</div><br/>

<div dir="rtl">
       مقدار متغیر <strong>python-version</strong> رو معادل نسخه پایتون استفاده شده قرار میده.
</div><br/>

<div dir="rtl">
     <strong>name: install dependencies</strong>
</div><br/>

<div dir="rtl">
       اسم <strong>step </strong>بعدی رو تعیین میکنه.
</div><br/>

<div dir="rtl">
       <strong>run</strong>
</div><br/>

<div dir="rtl">
      دستوری که میخوایم تو <strong>shell </strong>اجرا بشه رو اینجا مینویسیم. هر دستوری میتونه باشه فقط باید تو سیستم عامل انتخابی معتبر باشه.
</div><br/>

<div dir="rtl">
       اینجا اول <strong>pip </strong>رو نصب و آپگرید میکنیم. بعد هر بار قستمی از فایل نوشته شده در <a href="https://vrgl.ir/oAqXx" target="_blank" rel="noopener nofollow">بخش اول این مقاله</a> رو اجرا میکنیم.
</div><br/>

<hr style="background-color: floralwhite;" />

<br/><div dir="rtl">
    تو این پست ایرانی پای دیدید که چجوری یک <strong>workflow </strong>تو گیت هاب راه بندازیم و با استفاده از اون کارای تکراری و خسته کننده مون رو به صورت خودکار انجام بدیم، تا هم تو وقت صرفه جویی کرده باشیم هم احتمال اشتباه کردن رو پایین آورده باشیم. با استفاده از این سرویسهای کاربردی گیت هاب کارای خیلی زیادی میشه انجام داد. خوبه که بلد باشید، حتما به کارتون میاد.
</div><br/>

<div dir="rtl">
    امیدوارم از این پست دو بخشی استفاده کافی رو برده باشید. اگه شما هم مثل ما از خوندن مطالب پایتونی لذت میبرید، خوشحال میشیم ما رو تو شبکه های اجتماعی دنبال و در صورت داشتن انتقاد و پیشنهاد ما رو ازش مطلع کنید. لایک هم یادتون نره :)<
</div><br/>

<div dir="rtl">
    <strong>اگه با فعال سازی و نحوه استفاده از <a href="https://pages.github.com/" target="_blank" rel="noopener nofollow">github pages</a> مشکلی دارید، توصیه میشه به این <a href="https://www.instagram.com/p/CUhvC00oACZ/?utm_source=ig_web_copy_link" target="_blank" rel="noopener nofollow">پست اینستاگرام</a>، ایرانی پای سر بزنید.</strong>
</div><br/>

<hr style="background-color: floralwhite;" />

<br/><div dir="rtl">
    <h4><strong>حمایت از ما</strong></h4>
</div><br/>

<div dir="rtl">
    با دنبال کردن ما در
</div>

<div dir="rtl">
    <ul><li><a href="https://github.com/iranipy" target="_blank" rel="noopener nofollow">github</a></li><li><a href="https://t.me/iranipy" target="_blank" rel="noopener nofollow">telegram</a></li><li><a href="https://instagram.com/iranipy" target="_blank" rel="noopener nofollow">instagram</a> </li><li><a href="https://twitter.com/_iranipy" target="_blank" rel="noopener nofollow">twitter</a> </li></ul>
</div>

<div dir="rtl">
    یا کمک مالی به مبلغ دلخواه از طریق <a href="https://idpay.ir/iranipy" target="_blank" rel="noopener nofollow">درگاه آیدی پی</a> میتونید از ما حمایت کنید.
</div>

<div dir="rtl">
    <strong>وب سایت (به زودی):</strong> <a href="https://iranipy.ir/" target="_blank" rel="noopener nofollow">iranipy.ir</a>
</div>